This notebook will focus on reading the .csv data from our dataset, inspecting the data

In [4]:
import pandas as pd

DATA_PATH = "data/"
PICKLE_PATH = "pickled_data/"

### Play Data
This set stores information about individual plays or events within a game. There are different event types corresponding to the moments of statistical interest within a hockey game. Some event types are associated with x, y coordinates giving location on the ice.

In [ ]:
game_plays = pd.read_csv(DATA_PATH+"game_plays.csv")
game_plays.info()

In [ ]:
null_plays = game_plays.isna().sum()
null_plays

There are some significant missing data at a glance, however we know that this by design. Certain events are missing team info because they are "neutral" and don't pertain to a particular team. X, Y coords are not recorded for every event type, and secondaryType is only recorded for shot events.

In [ ]:
game_plays['event'].value_counts()

Example of an event type without certain data by design:

In [ ]:
game_plays[game_plays['event'] == 'Shootout Complete']

In [ ]:
duplicate_plays = game_plays.groupby(['play_id'])['game_id'].count() > 1
duplicate_plays.value_counts()

There is some duplication based on the play_id. We should attempt to drop duplicates with the same play_id, in the same game, happening at the same exact time.

In [48]:
subset_columns = ['play_id', 'game_id', 'dateTime']
game_plays = game_plays.drop_duplicates(subset=subset_columns, keep="first")

In [ ]:
duplicate_plays = game_plays.groupby(subset_columns)['game_id'].count() > 1
duplicate_plays.value_counts()

With no more duplicates, we can take a look at the data we're most interested in, shots and goals

In [ ]:
shots = game_plays[game_plays['event'] == "Shot"]
shots.head()

In [ ]:
shots['secondaryType'].value_counts()

In [ ]:
shots['secondaryType'].isna().sum()

There are very few missing shots, we can fill them with the mode 'Wrist Shot'

In [68]:
shot_mask = game_plays['event'] == 'Shot'
game_plays.loc[shot_mask, 'secondaryType'] = game_plays.loc[shot_mask, 'secondaryType'].fillna('Wrist Shot')

In [ ]:
game_plays[shot_mask]['secondaryType'].isna().sum()

With the missing values for secondaryType populated, this data is ready to go!

In [72]:
game_plays.to_pickle(PICKLE_PATH + "game_plays")

### Game Data
This table gives data about the game itself. Each game has an ID, the season it was played in, the teams that participated, the score and outcome, and venue data

In [ ]:
games = pd.read_csv(DATA_PATH + "game.csv")
games.info()

In [ ]:
games.shape

In [ ]:
games['season'].unique()

We have no null data in the games set, let's check for duplicates.

In [ ]:
duplicate_games = games.groupby(['game_id'])['season'].count() > 1
duplicate_games.value_counts()

Looks like there is some duplication. We can say for sure that no two games should have the same id, season, venue, and teams involved. If all those factors were the same, the game row is certainly duplicate

In [6]:
subset_columns = ['game_id', 'season', 'venue', 'away_team_id', 'home_team_id']
games = games.drop_duplicates(subset=subset_columns, keep="first")

In [ ]:
games.shape

In [8]:
games.to_pickle(PICKLE_PATH+"games")

### Game Teams stats
This table gives aggregate statistics and other team data for each game. Goals, shots, hits, penalties, faceoff win %, etc.

In [ ]:
game_teams = pd.read_csv(DATA_PATH+"game_teams_stats.csv")
game_teams.info()

In [ ]:
game_teams.shape

There is some null data in faceoff win %, giveaways, takeaways, hits. Hits were not always a tracked stat and were only begun to be measured at some point during the timeline of this data, so we are expecting some missing values. For the others, the data isn't in areas we anticipate investigating heavily, so we will forego imputing any values for now.

Let's move on to inspecting for duplication.

In [ ]:
team_duplicates = game_teams.groupby(['game_id', 'team_id'])['won'].count() > 1
team_duplicates.value_counts()

No game should have the same id with multiple entries for two teams of the same id. Ex. for each game there should be two entries, one for each team.

In [13]:
subset_columns = ['game_id', 'team_id']
game_teams = game_teams.drop_duplicates(subset=subset_columns, keep="first")

In [ ]:
game_teams.shape

Duplicates dropped, and a consistent result mathematically with our games set. 

In [18]:
game_teams.to_pickle(PICKLE_PATH + "game_teams")

### Game Plays Players
This table is an intermediate mapping table for matching game_plays to player_info. It contains a play_id, game_id, and player_id along with playerType

In [ ]:
game_plays_players = pd.read_csv(DATA_PATH + "game_plays_players.csv")
game_plays_players.info()

Let's inspect what information is stored in the playerType field

In [ ]:
game_plays_players['playerType'].value_counts()

It looks like the playerType is informative to the type of event happening in the play. Let's check for duplicate values.

In [ ]:
duplicate_play_players = game_plays_players.groupby(['play_id', 'game_id', 'player_id'])['playerType'].count() > 1
duplicate_play_players.value_counts()

In [24]:
subset_columns = ['play_id', 'game_id', 'player_id']
game_plays_players = game_plays_players.drop_duplicates(subset=subset_columns, keep="first")

In [ ]:
game_plays_players.shape

In [26]:
game_plays_players.to_pickle(PICKLE_PATH + 'game_plays_players')

### Player Info 
This table contains biographical information about each player, including first and last name, nationality, birth city, position, birthday.

In [ ]:
player_info = pd.read_csv(DATA_PATH + "player_info.csv")
player_info.info()

There are no null values in fields of interest for our group. First, Last, birthday  (for age), and position are the only data points we plan to use.

In [ ]:
duplicate_players = player_info.groupby(['player_id', 'firstName', 'lastName'])['nationality'].count() > 1
duplicate_players.value_counts()

There are no duplicate players in the set.

In [ ]:
all_stars = pd.read_csv(DATA_PATH + 'all_star_data.csv')
all_stars.shape

In [57]:
player_info['fullName'] = player_info['firstName'] + ' ' + player_info['lastName']

In [ ]:
merged = pd.merge(all_stars, player_info, left_on='Player', right_on='fullName', how='left')
merged[merged['player_id'].isna()]

In [ ]:
player_seasons = merged.groupby('player_id')['Season'].apply(list).reset_index()
player_info = pd.merge(player_info, player_seasons, on='player_id', how='left').rename(columns={'Season': 'allStarSeasons'})
player_info

In [31]:
player_info.to_pickle(PICKLE_PATH + "player_info")